In [1]:
!pip install transformers[torch]
!pip install accelerate -U
!pip install datasets
!pip install transformers
!pip install torchaudio==0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
Looking in links: https://download.pytorch.org/whl/cu113/torch_st

In [6]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install evaluate
from datasets import load_dataset, Audio
from evaluate import load


In [10]:
common_voice_train = load_dataset("mozilla-foundation/common_voice_17_0", "ne-NP", split="validated+other")

n_shards.json:   0%|          | 0.00/17.5k [00:00<?, ?B/s]

ne-NP_train_0.tar:   0%|          | 0.00/6.12M [00:00<?, ?B/s]

ne-NP_dev_0.tar:   0%|          | 0.00/4.63M [00:00<?, ?B/s]

ne-NP_test_0.tar:   0%|          | 0.00/5.91M [00:00<?, ?B/s]

ne-NP_other_0.tar:   0%|          | 0.00/15.3M [00:00<?, ?B/s]

...


Reading metadata...: 595it [00:00, 50725.86it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 64it [00:00, 53708.57it/s]


Generating validated split: 0 examples [00:00, ? examples/s]


Reading metadata...: 742it [00:00, 36098.24it/s]


In [8]:
# Load the WER metric
wer_metric = load("wer")

In [12]:
# This cell basically splits the dataset into three subsets. 80% goes to the training set, 10% goes to the validation set and 10% goes to the test set
from datasets import load_dataset

# Define proportions for training, validation, and test sets
train_ratio = 0.8  # 80% for training
validation_ratio = 0.1  # 10% for validation
test_ratio = 0.1  # 10% for testing
common_voice_train = load_dataset("mozilla-foundation/common_voice_17_0", "ne-NP", split="validated+other")
# Calculate sizes based on ratios
num_samples = len(common_voice_train)
print(f'Original: {num_samples}')
train_size = int(train_ratio * num_samples)
validation_size = int(validation_ratio * num_samples)
test_size = num_samples - train_size - validation_size

# Split the dataset
train_dataset = common_voice_train.select(range(0, train_size))
common_voice_dev = common_voice_train.select(range(train_size, train_size + validation_size))
common_voice_test = common_voice_train.select(range(train_size + validation_size, train_size + validation_size + test_size))

common_voice_train = train_dataset

# Print the sizes of each dataset
print(f"Training dataset size: {len(common_voice_train)} samples")
print(f"Validation dataset size: {len(common_voice_dev)} samples")
print(f"Test dataset size: {len(common_voice_test)} samples")

Original: 1337
Training dataset size: 1069 samples
Validation dataset size: 133 samples
Test dataset size: 135 samples


In [13]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voiec_dev = common_voice_dev.remove_columns(['accent', 'age', 'client_id', 'down_votes', 'gender', 'locale', 'segment', 'up_votes'])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [14]:
import re
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\']'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_remove_regex, '', batch["sentence"]).lower()
    return batch

In [15]:
common_voice_train = common_voice_train.map(remove_special_characters)
common_voice_dev = common_voice_dev.map(remove_special_characters)
common_voice_test = common_voice_test.map(remove_special_characters)

Map:   0%|          | 0/1069 [00:00<?, ? examples/s]

Map:   0%|          | 0/133 [00:00<?, ? examples/s]

Map:   0%|          | 0/135 [00:00<?, ? examples/s]

In [43]:
from transformers import AutoModelForCTC, Wav2Vec2Processor

model = AutoModelForCTC.from_pretrained("iamtangsang/test-wav2vec2-large-xls-r-300m-nepali-openslr")
processor = Wav2Vec2Processor.from_pretrained("iamtangsang/test-wav2vec2-large-xls-r-300m-nepali-openslr")

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

## Below is for Common Voice


In [17]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_dev = common_voice_dev.cast_column("audio", Audio(sampling_rate=16_000))

In [18]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

print(common_voice_train[rand_int]["sentence"])
ipd.Audio(data=common_voice_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)

आज राती पानी परे त भोलि धान रोप्न एकदम सजिलो हुन्थ्यो ।


In [19]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]['sentence'])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])

Target text: आमा लाई मासु पकाउन भन ।
Input array shape: (47808,)
Sampling rate: 16000


In [20]:
common_voice_test[0]['audio']['sampling_rate']

16000

In [21]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [22]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names)
common_voice_dev = common_voice_dev.map(prepare_dataset, remove_columns=common_voice_dev.column_names)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names)

Map:   0%|          | 0/1069 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/133 [00:00<?, ? examples/s]

Map:   0%|          | 0/135 [00:00<?, ? examples/s]

In [24]:
# Define proportions for training, validation, and test sets
train_ratio = 0.8  # 80% for training
validation_ratio = 0.1  # 10% for validation
test_ratio = 0.1  # 10% for testing
common_voice_train_evaluation = load_dataset("mozilla-foundation/common_voice_17_0", "ne-NP", split="validated+other")
# Calculate sizes based on ratios
num_samples = len(common_voice_train_evaluation)
print(f'Original: {num_samples}')
train_size = int(train_ratio * num_samples)
validation_size = int(validation_ratio * num_samples)
test_size = num_samples - train_size - validation_size

# Split the dataset
train_dataset = common_voice_train_evaluation.select(range(0, train_size))
common_voice_dev_evaluation = common_voice_train_evaluation.select(range(train_size, train_size + validation_size))
common_voice_test_evaluation = common_voice_train_evaluation.select(range(train_size + validation_size, train_size + validation_size + test_size))

common_voice_train_evaluation = train_dataset

# Print the sizes of each dataset
print(f"Training dataset size: {len(common_voice_train_evaluation)} samples")
print(f"Validation dataset size: {len(common_voice_dev_evaluation)} samples")
print(f"Test dataset size: {len(common_voice_test_evaluation)} samples")

Original: 1337
Training dataset size: 1069 samples
Validation dataset size: 133 samples
Test dataset size: 135 samples


In [25]:
common_voice_train_evaluation = common_voice_train_evaluation.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test_evaluation = common_voice_test_evaluation.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_dev_evaluation = common_voice_dev_evaluation.cast_column("audio", Audio(sampling_rate=16_000))

In [26]:
import torch
import random


In [30]:

random_int = random.randint(0, len(common_voice_dev)-1)
input_dict = processor(common_voice_dev[random_int]["input_values"], return_tensors="pt", padding=True)

logits = model(input_dict.input_values).logits

pred_ids = torch.argmax(logits, dim=-1)[0]

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


## Comparison with other's model

In [99]:
# model = AutoModelForCTC.from_pretrained("spktsagar/wav2vec2-large-xls-r-300m-nepali-openslr")
# processor = Wav2Vec2Processor.from_pretrained("spktsagar/wav2vec2-large-xls-r-300m-nepali-openslr")
model = AutoModelForCTC.from_pretrained("Strange18/wav2vec2-nepali-asr")
processor = Wav2Vec2Processor.from_pretrained("Strange18/wav2vec2-nepali-asr")
model_tang = AutoModelForCTC.from_pretrained("iamtangsang/test-wav2vec2-large-xls-r-300m-nepali-openslr")
processor_tang = Wav2Vec2Processor.from_pretrained("iamtangsang/test-wav2vec2-large-xls-r-300m-nepali-openslr")

In [85]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [100]:
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2CTCTokenizer, Wav2Vec2ForCTC
model_cv =  Wav2Vec2ForCTC.from_pretrained('/content/drive/MyDrive/checkpoint-800')
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("/content/drive/MyDrive/checkpoint-800", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)
processor_cv = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [120]:
import IPython.display as ipd
random_int = random.randint(0, len(common_voice_dev_evaluation)-1)
# random_int = 800
# For Common Voice
input_dict_cv = processor_cv(common_voice_dev[random_int]["input_values"], return_tensors="pt", padding=True)
logits_cv = model_cv(input_dict_cv.input_values).logits

# For OpenSLR
input_dict_open_slr = processor_tang(common_voice_dev[random_int]['input_values'], return_tensors='pt', padding=True)
logits_open_slr = model_tang(input_dict_open_slr.input_values).logits

#For Sagar
input_dict_sagar = processor(common_voice_dev[random_int]["input_values"], return_tensors="pt", padding=True)
logits_sagar = model(input_dict_sagar.input_values).logits

#For Sagar
pred_ids_sagar = torch.argmax(logits_sagar, dim=-1)[0]

#For common Voice
pred_ids_cv = torch.argmax(logits_cv, dim=-1)[0]

#For OpenSLR
pred_ids_open_slr = torch.argmax(logits_open_slr, dim=-1)[0]

print("Prediction:")
print("Prediction From Sagar's model: ")
print(processor.decode(pred_ids_sagar))
print("Prediction From our Common Voice model: ")
print(processor_cv.decode(pred_ids_cv))
print("Prediction from our OpenSLR model: ")
print(processor_tang.decode(pred_ids_open_slr))


print("\nReference:")
print(common_voice_dev_evaluation[random_int]["sentence"].lower())
ipd.Audio(data=common_voice_dev_evaluation[random_int]['audio']['array'], autoplay=True, rate=16000)


It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Prediction:
Prediction From Sagar's model: 
बाँदरले एकदम मेठो भुने गरी दही च्युउराएर चिनी मुछेको थियो
Prediction From our Common Voice model: 
बादरले एकदम मीठो हुनेगरि दहि चिउरार चिनी मुरषेको थियो ।
Prediction from our OpenSLR model: 
बादरले एकदममिठो हुनीगरदहि ुउरार चिनि मुचेको थियो

Reference:
बाँदरले एकदम मिठो हुने गरी दही, चिउरा र चिनी मुछेको थियो ।


### From my model

In [53]:
model = AutoModelForCTC.from_pretrained("iamtangsang/test-wav2vec2-large-xls-r-300m-nepali-openslr")
processor = Wav2Vec2Processor.from_pretrained("iamtangsang/test-wav2vec2-large-xls-r-300m-nepali-openslr")

In [54]:
import IPython.display as ipd
random_int = random.randint(0, len(common_voice_train)-1)
random_int = 69
input_dict = processor(common_voice_train[random_int]["input_values"], return_tensors="pt", padding=True)
logits = model(input_dict.input_values).logits

pred_ids = torch.argmax(logits, dim=-1)[0]

print("Prediction:")
print(processor.decode(pred_ids))

print("\nReference:")
print(common_voice_train_evaluation[random_int]["sentence"].lower())
ipd.Audio(data=common_voice_train_evaluation[random_int]['audio']['array'], autoplay=True, rate=16000)


It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Prediction:
मैले माछा पोखरीतिरछु

Reference:
म अहिले माछापोखरी तिर छु ।


In [35]:
print('Hello World! Thank You!!')

Hello World! Thank You!!


# Inference From a File

In [ ]:
from IPython.display import Audio
from scipy.io import wavfile
import numpy as np

In [ ]:
from transformers import Wav2Vec2ForCTC

# tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("/content/drive/MyDrive/Nepali_ASR/results/checkpoint-800", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
model = Wav2Vec2ForCTC.from_pretrained('/content/drive/MyDrive/Nepali_ASR/results/checkpoint-800/')

In [ ]:
from transformers import Wav2Vec2Tokenizer
tokenizer = Wav2Vec2Tokenizer.from_pretrained("/content/drive/MyDrive/Nepali_ASR/results/checkpoint-800", unk_token='[UNK]', pad_token='[PAD]', word_delimiter_token='|')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:720: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(


In [ ]:
file_name = '/content/drive/MyDrive/Nepali_ASR/test-3.mp3'

In [ ]:
from pydub import AudioSegment

# Load MP3 file
audio = AudioSegment.from_mp3(file_name)

# Export as WAV
wav_file = "/content/drive/MyDrive/Nepali_ASR/test-3-wav.wav"
audio.export(wav_file, format="wav")


<_io.BufferedRandom name='/content/drive/MyDrive/Nepali_ASR/test-3-wav.wav'>

In [ ]:
Audio(wav_file)

In [ ]:
data = wavfile.read(wav_file)
framerate = data[0]
sounddata = data[1]
time = np.arange(0,len(sounddata))/framerate
print('Sample rate:',framerate,'Hz')
print('Total time:',len(sounddata)/framerate,'s')


Sample rate: 48000 Hz
Total time: 4.056 s


In [ ]:
import soundfile as sf
import librosa

In [ ]:
input_audio, _ = librosa.load(wav_file,
                              sr=16000)

In [ ]:
# input_values = tokenizer(input_audio, return_tensors="pt", padding=True).input_values
input_values = tokenizer(input_audio, return_tensors='pt').input_values
logits = model(input_values).logits
predicted_ids = torch.argmax(logits, dim=-1)
transcription = tokenizer.batch_decode(predicted_ids)[0]

In [ ]:
print(f'Prediction:\n{transcription}')

Prediction:
म अहिले मैजार र्जेर बोति मोती राखेको छु।
